In [199]:
import os
import astropy.io.fits as fits
import datetime as dt
import gzip
import re
import glob
from pprint import pprint as pp

import requests
import json
import dateutil as du

In [200]:
root_dir = "/home/pi/GoogleDrive/mydrive/Data/astro/suhora_logs/logs_test/M35/"
ALLOWED_EXT = ['.gz', '.bz2']
forbidden_keys = ['HISTORY', 'COMMENT']
DATE_REGEX = '^\d{4}\-(0[1-9]|1[012])\-(0[1-9]|[12][0-9]|3[01])$'

In [201]:
START_DATETIME = '1019-01-27T12:06:21'

In [202]:
def get_dirs_to_walk(root_dir, START_DATETIME):
    folders_in_root = os.listdir(root_dir)
    
    start_datetime = du.parser.isoparse(START_DATETIME)
    start_date = start_datetime.date()
    if start_datetime.hour >= 12:
        start_date = start_date + dt.timedelta(days=1)
        
    p = re.compile(DATE_REGEX) 
    
    dirs_to_walk = []
    for _d in os.listdir(root_dir):
        if (p.match(_d) and \
                dt.datetime.strptime(_d, '%Y-%m-%d').date() >= start_date):
            dirs_to_walk.append(os.path.join(root_dir, _d))
            
    return dirs_to_walk


def get_files(_dir):
    
    files_in_folder = []
    for r, d, f in os.walk(_dir):
        for file in f:
            files_in_folder.append(os.path.join(r, file))
    
    files_to_open = []
    for _file in files_in_folder:
        if os.path.isfile(_file):
            if os.path.splitext(_file)[-1] in ALLOWED_EXT:
                files_to_open.append(_file)
    
    return files_to_open


def get_folder_data(files_to_open):
    folder_data = []
    for f in files_to_open:
        with gzip.open(f, 'rb') as f_in:
            hdr = dict(fits.getheader(f))
            
            obs_datetime = dt.datetime.strptime(
                hdr['DATE-OBS'] + 'T' + hdr['TIME-OBS'],
                '%Y-%m-%dT%H:%M:%S.%f').isoformat()
            
            object_name = hdr['OBJECT']
            observer = hdr['OBSERVER']
            color_filter = hdr['FILTER']
            exptime = hdr['EXPTIME']
            
        row = {
            'obs_datetime': obs_datetime,
            'object_name': object_name,
            'observer': observer,
            'color_filter': color_filter,
            'exptime': exptime,
        }
        folder_data.append(row)
        
    folder_data = sorted(folder_data, key=lambda x: x['obs_datetime'])
    
    return folder_data


def get_grouped_folder_data(folder_data):
    results = []
    target = {}
    n_frames = 0
    prev_name = None

    for frame in folder_data:
        if frame['object_name'] != prev_name:
            n_frames = 0
            if target:
                target['datetime_end'] = frame['obs_datetime']
                target['colorfilters'] = [
                    {'name': c} for c in target['colorfilters']
                ]
                results.append(target)
            target = {}
            prev_name = frame['object_name']
            target['name'] = frame['object_name']
            target['datetime_start'] = frame['obs_datetime']
            target['observer'] = frame['observer']
            target['colorfilters'] = []
            target['total_exposure_time'] = 0
            target['number_of_frames'] = 0
            target['telescope'] = 'M35'

        target['number_of_frames'] += 1    
        target['total_exposure_time'] += float(frame.get('exptime', 0))
        if frame['color_filter'] not in target['colorfilters']:
            target['colorfilters'].append(frame['color_filter'])

    return results
    


In [203]:
dirs_to_walk = sorted(get_dirs_to_walk(root_dir, START_DATETIME))
data_to_send = []
for _dir in dirs_to_walk:
    print('#'*9)
    pp(_dir)
    folder_files = get_files(_dir)
    folder_data = get_folder_data(folder_files)
    grouped_folder_data = get_grouped_folder_data(folder_data)
    data_to_send.append(grouped_folder_data)
    pp(grouped_folder_data)
    print('#'*9)

#########
'/home/pi/GoogleDrive/mydrive/Data/astro/suhora_logs/logs_test/M35/2017-04-02'
[{'colorfilters': [{'name': 'R'},
                   {'name': 'I'},
                   {'name': 'V'},
                   {'name': 'B'},
                   {'name': 'RS'},
                   {'name': 'IS'},
                   {'name': 'GS'}],
  'datetime_end': '2017-04-02T17:52:31.270000',
  'datetime_start': '2017-04-02T17:38:55.850000',
  'name': 'flat',
  'number_of_frames': 115,
  'observer': 'MS',
  'telescope': 'M35',
  'total_exposure_time': 509.0},
 {'colorfilters': [{'name': 'B'}],
  'datetime_end': '2017-04-02T18:11:32.690000',
  'datetime_start': '2017-04-02T17:52:31.270000',
  'name': 'dark',
  'number_of_frames': 60,
  'observer': 'MS',
  'telescope': 'M35',
  'total_exposure_time': 850.0},
 {'colorfilters': [{'name': 'B'}],
  'datetime_end': '2017-04-02T18:13:20.820000',
  'datetime_start': '2017-04-02T18:11:32.690000',
  'name': 'bias',
  'number_of_frames': 50,
  'observer': 'MS',
  

In [166]:
for folder_ds in data_to_send:
    for ds in folder_ds:
        response = requests.post('http://127.0.0.1:8000/targets/', auth=('admin', 'haslobrzmi'),
                  data=json.dumps(ds),
                  headers={'content-type': 'application/json'})

        print(response)
        print(response.content)

<Response [201]>
b'{"datetime_start":"2017-04-02T17:38:55.850000Z","datetime_end":"2017-04-02T17:52:31.270000Z","observer":"MS","name":"flat","ra":null,"dec":null,"note":"","telescope":"M35","colorfilters":[{"name":"B"},{"name":"V"},{"name":"R"},{"name":"I"},{"name":"GS"},{"name":"IS"},{"name":"RS"}],"total_exposure_time":"509.00","number_of_frames":115}'
<Response [201]>
b'{"datetime_start":"2017-04-02T17:52:31.270000Z","datetime_end":"2017-04-02T18:11:32.690000Z","observer":"MS","name":"dark","ra":null,"dec":null,"note":"","telescope":"M35","colorfilters":[{"name":"B"}],"total_exposure_time":"850.00","number_of_frames":60}'
<Response [201]>
b'{"datetime_start":"2017-04-02T18:11:32.690000Z","datetime_end":"2017-04-02T18:13:20.820000Z","observer":"MS","name":"bias","ra":null,"dec":null,"note":"","telescope":"M35","colorfilters":[{"name":"B"}],"total_exposure_time":"0.00","number_of_frames":50}'
<Response [201]>
b'{"datetime_start":"2017-04-02T18:13:20.820000Z","datetime_end":"2017-04-0

In [188]:
timeout = 5
url = 'http://127.0.0.1:8000/stats/targets/z60'
try:
    response = requests.get(url, timeout=timeout,
                            auth=('admin', 'haslobrzmi'),
                  headers={'content-type': 'application/json'})
    print('connected')
except (requests.ConnectionError, requests.Timeout) as exception:
    response = None
    print('No connection!')
    
print(response)
print(response.content)
jr = json.loads(response.content)

connected
<Response [400]>
b'"The 6 telescope was not found in the DB"'


In [197]:
r = requests.Response()

In [198]:
r.ok

TypeError: '<=' not supported between instances of 'int' and 'NoneType'